In [ ]:
import os
import collections
import pandas as pd
import numpy as np
import functools
import matplotlib.pyplot as plt
import cv2
import numpy as np

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import xml.etree.ElementTree as ET

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import torch
import torchvision

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

from torch.utils.data import DataLoader, Dataset
from torch.utils.data import SequentialSampler

In [ ]:
# IMAGE_DIR = "Data/Train_Images"
IMAGE_DIR = "Data/Batch"
DATA = "Data/Train.csv"

In [ ]:
def image_data_set_to_df(data, image_data):
    transormed_data = []
    for image_filename in image_data:
        image_id = image_filename.replace(".jpg", "")
        dataframes = data.loc[data["Image_ID"] == image_id]
        for index, df in dataframes.iterrows():
            transormed_data.append([
                df["Image_ID"],
                df["class"],
                df["xmin"],
                df["ymin"],
                df["xmin"] + df["width"],
                df["ymin"] + df["height"]
            ])
    transormed_df = pd.DataFrame(data=transormed_data, columns=['img_id', 'names', 'xmin', 'ymin', 'xmax', 'ymax'])
    return transormed_df

In [ ]:
data = pd.read_csv(DATA)
train_image_filenames = os.listdir(IMAGE_DIR)

df = image_data_set_to_df(data=data, image_data=train_image_filenames)

In [ ]:
df['names'].value_counts()

In [ ]:
df.head

In [ ]:
enc = preprocessing.LabelEncoder()
df['labels'] = enc.fit_transform(df['names'])
df['labels'] = np.stack(df['labels'][i]+1 for i in range(len(df['labels']))) 

In [ ]:
classes = df[['names','labels']].value_counts()
classes

In [ ]:
classes= {
    1:'fruit_brownspot',
    2:'fruit_healthy',
    3:'fruit_woodiness'
}

In [ ]:
train_df, valid_df = train_test_split(df, test_size=0.2)

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))


class PassionFruitDataset(Dataset):
    
    def __init__(self, dataframe, image_dir, transforms=None):
        super().__init__()
        
        self.image_ids = dataframe['img_id'].unique()
        self.df = dataframe
        self.image_dir = image_dir
        self.transforms = transforms
    
    def __getitem__(self, index: int):
        image_id = self.image_ids[index]
        records = self.df[self.df['img_id'] == image_id]
        
        image = cv2.imread(f'{self.image_dir}/{image_id}.jpg', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        rows, cols = image.shape[:2]
        
        boxes = records[['xmin', 'ymin', 'xmax', 'ymax']].values
        
       
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        area = torch.as_tensor(area, dtype=torch.float32)
        
        label = records['labels'].values
        labels = torch.as_tensor(label, dtype=torch.int64)
        
        # suppose all instances are not crowd
        iscrowd = torch.zeros((records.shape[0],), dtype=torch.int64)
        
        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        # target['masks'] = None
        target['image_id'] = torch.tensor([index])
        target['area'] = area
        target['iscrowd'] = iscrowd
        
        if self.transforms:
            sample = {
                'image': image,
                'bboxes': target['boxes'],
                'labels': labels
            }
            sample = self.transforms(**sample)
            image = sample['image']
            
            target['boxes'] = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes'])))).permute(1,0)
            
            return image, target
        
    def __len__(self) -> int:
        return self.image_ids.shape[0]

In [ ]:
def get_transform_train():
    return A.Compose([
        A.HorizontalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.2),
        ToTensorV2(p=1.0)
    ], bbox_params={'format':'pascal_voc', 'label_fields': ['labels']})

def get_transform_valid():
    return A.Compose([
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields':['labels']})

In [ ]:
train_dataset = PassionFruitDataset(train_df, IMAGE_DIR , get_transform_train())
valid_dataset = PassionFruitDataset(valid_df, IMAGE_DIR, get_transform_valid())

In [ ]:
# split the dataset in train and test set
indices = torch.randperm(len(train_dataset)).tolist()

train_data_loader = DataLoader(
    train_dataset,
    batch_size=4,
    shuffle=True,
    num_workers=4,
    collate_fn=collate_fn
)

valid_data_loader = DataLoader(
    valid_dataset,
    batch_size=4,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
images, targets= next(iter(train_data_loader))
images = list(image.to(device) for image in images)
targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

plt.figure(figsize=(20,20))
for i, (image, target) in enumerate(zip(images, targets)):
    plt.subplot(2,2, i+1)
    boxes = targets[i]['boxes'].cpu().numpy().astype(np.int32)
    sample = images[i].permute(1,2,0).cpu().numpy()
    names = targets[i]['labels'].cpu().numpy().astype(np.int64)
    for i,box in enumerate(boxes):
        cv2.rectangle(sample,
                      (box[0], box[1]),
                      (box[2], box[3]),
                      (0, 0, 220), 2)
        cv2.putText(sample, classes[names[i]], (box[0],box[1]+15),cv2.FONT_HERSHEY_COMPLEX ,0.5,(0,220,0),1,cv2.LINE_AA)  

    plt.axis('off')
    plt.imshow(sample)
    

In [ ]:
# load a model; pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

In [ ]:
num_classes = 4

# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features

# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

In [ ]:
model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [ ]:
# !pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

In [ ]:
# !git clone https://github.com/pytorch/vision.git
# !cd vision;cp references/detection/utils.py ../;cp references/detection/transforms.py ../;cp references/detection/coco_eval.py ../;cp references/detection/engine.py ../;cp references/detection/coco_utils.py ../

In [ ]:
def _summarize(p, s, ap=1, iouThr=None, areaRng='all', maxDets=100):
    # p = self.params
    iStr = ' {:<18} {} @[ IoU={:<9} | area={:>6s} | maxDets={:>3d} ] = {:0.3f}'
    titleStr = 'Average Precision' if ap == 1 else 'Average Recall'
    typeStr = '(AP)' if ap==1 else '(AR)'
    iouStr = '{:0.2f}:{:0.2f}'.format(p.iouThrs[0], p.iouThrs[-1]) \
        if iouThr is None else '{:0.2f}'.format(iouThr)

    aind = [i for i, aRng in enumerate(p.areaRngLbl) if aRng == areaRng]
    mind = [i for i, mDet in enumerate(p.maxDets) if mDet == maxDets]
    if ap == 1:
        # dimension of precision: [TxRxKxAxM]
        # IoU
        if iouThr is not None:
            t = np.where(iouThr == p.iouThrs)[0]
            s = s[t]
        s = s[:,:,:,aind,mind]
    else:
        # dimension of recall: [TxKxAxM]
        if iouThr is not None:
            t = np.where(iouThr == p.iouThrs)[0]
            s = s[t]
        s = s[:,:,aind,mind]
    if len(s[s>-1])==0:
        mean_s = -1
    else:
        mean_s = np.mean(s[s>-1])
    print(iStr.format(titleStr, typeStr, iouStr, areaRng, maxDets, mean_s))
    return mean_s

In [ ]:
from engine import train_one_epoch, evaluate
import utils
import wandb

In [ ]:
config_default = {         
    "batch_size": 8,          # input batch size for training (default: 64)
    "test_batch_size": 8,    # input batch size for testing (default: 1000)
    "epochs": 2,             # number of epochs to train (default: 10)
    "lr": 0.005,               # learning rate (default: 0.01)
    "momentum": 0.9,          # SGD momentum (default: 0.5) 
    "no_cuda": False,         # whether to disable CUDA training
    "seed": 42,               # random seed (default: 42)
    "log_interval": 1,      #how many batches to wait before logging in train/test loops
    "image_log_interval": 10,
    "decay": 0.0005
}

In [ ]:
def get_precision_and_recall_metrics(coco_evaluator):
    data = {}
    coco = coco_evaluator.coco_eval['bbox']
    data['metrics/mAP_0.5'] = _summarize(coco.params, coco.eval['precision'], 1, iouThr=.5, maxDets=coco.params.maxDets[2])
    data['metrics/mAP_0.5:0.95'] = _summarize(coco.params, coco.eval['precision'], 1, areaRng='large', maxDets=coco.params.maxDets[2])
    data["metrics/recall"] = _summarize(coco.params, coco.eval['recall'], 0, iouThr=.5, maxDets=coco.params.maxDets[1])
    return data
    

In [ ]:
def get_loss_data(metric_logger):
    d = {}
    data = {}
    for metric in metric_logger.meters.items():
        d[metric[0]] = metric[1].avg
    
    data['train/obj_loss'] = d['loss_objectness']
    data['train/box_loss'] = d['loss_box_reg']
    data['train/cls_loss'] = d['loss_classifier']
    data['loss'] = d['loss']
    return data

In [ ]:
wandb.init(config=config_default, project="GWD-fasterRCNN", api_key="dbfbcbefbdb1ccbef7d90bf547b9a11b9a696bd1")
wandb.login()

In [ ]:
wandb.init(config=config_default, project="GWD-fasterRCNN")

In [ ]:
# let's train it for 2 epochs
num_epochs = 1
for epoch in range(num_epochs):
    wand_data = {}
    # train for one epoch, printing every 10 iterations
    metric_logger = train_one_epoch(model, optimizer, train_data_loader, device, epoch, print_freq=10)
    metric_data = get_loss_data(metric_logger)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    coco_evaluator = evaluate(model, valid_data_loader, device=device)
    metric_data.update(get_precision_and_recall_metrics(coco_evaluator))
    wandb.log(metric_data)

In [ ]:
coco_evaluator.analyze()

In [ ]:
torch.save(model.state_dict(), 'faster_rcnn_state.pth')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 

all_precision =  evaluator.eval['precision']
all_precision =  evaluator.eval['precision']

pr_5 = all_precision[0, :, 0, 0, 2] # data for IoU@0.5
pr_7 = all_precision[4, :, 0, 0, 2] # data for IoU@0.7
pr_9 = all_precision[8, :, 0, 0, 2] # data for IoU@0.9

x = np.arange(0, 1.01, 0.01)
plt.plot(x, pr_5, label='IoU@0.5')
plt.plot(x, pr_7, label='IoU@0.7')
plt.plot(x, pr_9, label='IoU@0.9')

plt.show()


In [ ]:
# load  a model; pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False, pretrained_backbone=False)

WEIGHTS_FILE = "./faster_rcnn_state.pth"

num_classes = 4

# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features

# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# Load the traines weights
model.load_state_dict(torch.load(WEIGHTS_FILE))

model = model.to(device)

In [ ]:
def obj_detector(img):
    img = cv2.imread(img, cv2.IMREAD_COLOR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)
    img /= 255.0
    img = torch.from_numpy(img)
    img = img.unsqueeze(0)
    img = img.permute(0,3,1,2)
    
    model.eval()

    detection_threshold = 0.70
    
    img = list(im.to(device) for im in img)
    output = model(img)

    for i , im in enumerate(img):
        boxes = output[i]['boxes'].data.cpu().numpy()
        scores = output[i]['scores'].data.cpu().numpy()
        labels = output[i]['labels'].data.cpu().numpy()

        labels = labels[scores >= detection_threshold]
        boxes = boxes[scores >= detection_threshold].astype(np.int32)
        scores = scores[scores >= detection_threshold]

        boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
        boxes[:, 3] = boxes[:, 3] - boxes[:, 1]
    
    sample = img[0].permute(1,2,0).cpu().numpy()
    sample = np.array(sample)
    boxes = output[0]['boxes'].data.cpu().numpy()
    name = output[0]['labels'].data.cpu().numpy()
    scores = output[0]['scores'].data.cpu().numpy()
    boxes = boxes[scores >= detection_threshold].astype(np.int32)
    names = name.tolist()
    
    return names, boxes, sample

In [ ]:
pred_path = "Data/Batch"
pred_files = [os.path.join(pred_path,f) for f in os.listdir(pred_path)]

plt.figure(figsize=(20,60))
for i, images in enumerate(pred_files):
    if i > 19:break
    plt.subplot(10,2,i+1)
    names,boxes,sample = obj_detector(images)
    for i,box in enumerate(boxes):
        cv2.rectangle(sample,
                      (box[0], box[1]),
                      (box[2], box[3]),
                      (0, 220, 0), 2)
        cv2.putText(sample, classes[names[i]], (box[0],box[1]-5),cv2.FONT_HERSHEY_COMPLEX ,0.7,(220,0,0),1,cv2.LINE_AA)  

    plt.axis('off')
    plt.imshow(sample)
#     plt.savefig('save_image.png', bbox_inches='tight')  # if you want to save result